In [1]:
!pip install autogen-agentchat~=0.2 autogen psutil --quiet 
# !pip install -q langchain-openai termcolor langchain_community duckduckgo_search wikipedia openapi-python-client==0.12.3 langgraph langchain_experimental yfinance
!pip install -q langchain-openai termcolor langchain_community duckduckgo_search wikipedia openapi-python-client langgraph langchain_experimental openai --quiet
!pip install -U docker --quiet 


[notice] A new release of pip is available: 23.2.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.2.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


In [2]:
import os
import autogen
from typing import Literal

from pydantic import BaseModel, Field
from typing_extensions import Annotated
from typing import List, Optional
from datetime import datetime, timedelta
from dataclasses import dataclass
from autogen import ConversableAgent
from dotenv import load_dotenv

import autogen
from autogen.cache import Cache

DEEPSEEK_INFERENCE_SERVER_URL = "http://localhost:8989"
# MODEL_NAME = "deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B"
DEEPSEEK_MODEL_NAME = "deepseek-ai/DeepSeek-R1-Distill-Qwen-7B"

INFERENCE_SERVER_URL = "http://localhost:8000"
MODEL_NAME = "ibm-granite/granite-3.1-8b-base"
API_KEY= "alanliuxiang"


In [3]:
# Configuration for the vLLM endpoint
llm_config1 = {
    "config_list": [
        {
            "model": MODEL_NAME,
            "api_key": API_KEY,
            "base_url": f"{INFERENCE_SERVER_URL}/v1"
        }
    ],
    "cache_seed": None,
    "temperature": 0.01,
    "timeout": 600,
}

llm_config = {
    "config_list": [
        {
            "model": DEEPSEEK_MODEL_NAME,
            "api_key": API_KEY,
            "base_url": f"{DEEPSEEK_INFERENCE_SERVER_URL}/v1"
        }
    ],
    "cache_seed": None,
    "temperature": 0.01,
    "timeout": 600,
}

In [4]:
import os
from dotenv import load_dotenv
from autogen import AssistantAgent, UserProxyAgent, GroupChat, GroupChatManager


class ResearchAnalysisSystem:
    def __init__(self):
        self.llm_config = llm_config

        self.code_execution_config = {
            "work_dir": "research_output",
            "use_docker": False,
        }

        self.setup_agents()
        self.setup_group_chat()

    def setup_agents(self):
        # User Proxy for coordination
        self.user_proxy = UserProxyAgent(
            name="admin",
            human_input_mode="NEVER",
            code_execution_config=self.code_execution_config,
            system_message="Admin coordinating research analysis.",
        )

        # Specialized Agents
        self.research_agent = AssistantAgent(
            name="researcher",
            llm_config=self.llm_config,
            system_message="""Find and analyze research papers. Focus on:
            1. Identifying relevant papers
            2. Extracting key findings
            3. Categorizing applications""",
        )

        self.data_analyst = AssistantAgent(
            name="analyst",
            llm_config=self.llm_config,
            system_message="""Analyze research data to:
            1. Extract metrics
            2. Generate visualizations
            3. Identify trends""",
        )

        self.reviewer = AssistantAgent(
            name="reviewer",
            llm_config=self.llm_config,
            system_message="""Review findings for:
            1. Accuracy
            2. Completeness
            3. Relevance""",
        )

        self.content_writer = AssistantAgent(
            name="writer",
            llm_config=self.llm_config,
            system_message="""Synthesize findings into clear reports with:
            1. Executive summaries
            2. Detailed analysis
            3. Recommendations""",
        )

    def setup_group_chat(self):
        # Create agent group
        self.agents = [
            self.user_proxy,
            self.research_agent,
            self.data_analyst,
            self.reviewer,
            self.content_writer,
        ]

        # Initialize group chat
        self.group_chat = GroupChat(agents=self.agents, messages=[], max_round=50)

        # Set up manager
        self.manager = GroupChatManager(
            groupchat=self.group_chat, llm_config=self.llm_config
        )

    def research_pipeline(self, topic):
        # Initialize research tasks
        tasks = [
            f"Research papers on {topic}",
            "Extract and categorize applications",
            "Generate visualization of findings",
            "Prepare comprehensive report",
        ]

        results = []
        for task in tasks:
            # Each task is processed by the group
            result = self.user_proxy.initiate_chat(
                self.manager, message=task, summary_method="last_msg"
            )
            results.append(result)

        return results

    def analyze_article(self, article_content):
        # Article analysis pipeline
        analysis_tasks = [
            f"Analyze structure and coherence: {article_content}",
            "Review style and tone",
            "Verify factual accuracy",
            "Provide improvement suggestions",
            "Generate final publication readiness report",
        ]

        analysis_results = []
        for task in analysis_tasks:
            result = self.user_proxy.initiate_chat(
                self.manager, message=task, summary_method="last_msg"
            )
            analysis_results.append(result)

        return analysis_results



In [5]:
def main():
    # Initialize system
    system = ResearchAnalysisSystem()

    # Example usage
    topic = "machine learning in healthcare"
    research_results = system.research_pipeline(topic)

    # Article analysis
    with open("article.txt", "r") as file:
        article_content = file.read()

    analysis_results = system.analyze_article(article_content)

    # Print final results
    print("\nResearch Analysis Results:")
    for i, result in enumerate(research_results):
        print(f"\nTask {i+1} Results:")
        print(result)

    print("\nArticle Analysis Results:")
    for i, result in enumerate(analysis_results):
        print(f"\nAnalysis Task {i+1} Results:")
        print(result)




In [ ]:
main()

admin (to chat_manager):

Research papers on machine learning in healthcare

--------------------------------------------------------------------------------
[autogen.oai.client: 03-15 07:30:08] {652} WARNING - Model deepseek-ai/DeepSeek-R1-Distill-Qwen-7B is not found. The cost will be 0. In your config_list, add field {"price" : [prompt_price_per_1k, completion_token_price_per_1k]} for customized pricing.
[autogen.oai.client: 03-15 07:30:24] {652} WARNING - Model deepseek-ai/DeepSeek-R1-Distill-Qwen-7B is not found. The cost will be 0. In your config_list, add field {"price" : [prompt_price_per_1k, completion_token_price_per_1k]} for customized pricing.

Next speaker: researcher

[autogen.oai.client: 03-15 07:41:59] {652} WARNING - Model deepseek-ai/DeepSeek-R1-Distill-Qwen-7B is not found. The cost will be 0. In your config_list, add field {"price" : [prompt_price_per_1k, completion_token_price_per_1k]} for customized pricing.
researcher (to chat_manager):

Okay, so I need to find a